In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
blind = pd.read_csv("../../Data/Data_submission/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle
donnees_abiotiques = pd.read_csv("../../Data/Data_train/enviroTab_pa_train.csv", sep = ';')  #fichier csv données abiotiques des patchID
blind_abiotique = pd.read_csv("../../Data/Data_submission/enviroTab_pa_test.csv", sep = ';')
presences = pd.read_excel("../../Data/Data_train/Presences_Absences_train.xlsx") #fichier excel presences/absences

In [4]:
X1 = blind_abiotique[['dayOfYear', 'geoUncertaintyInM', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand',
       'silt', 'soc', 'landCov', 'Built1994', 'Lights1994', 'NavWater1994',
       'Built2009', 'Lights2009', 'NavWater2009', 'Popdensity1990',
       'Popdensity2010', 'Railways', 'Roads']].values
X2 = data_sans_doublons[['dayOfYear', 'geoUncertaintyInM', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand',
       'silt', 'soc', 'landCov', 'Built1994', 'Lights1994', 'NavWater1994',
       'Built2009', 'Lights2009', 'NavWater2009', 'Popdensity1990',
       'Popdensity2010', 'Railways', 'Roads']].values

NameError: name 'data_sans_doublons' is not defined

In [ ]:
colonnes_avec_nan = blind_abiotique.columns[blind_abiotique.isna().any()]
colonnes_avec_nan

In [17]:
data_sans_doublons = presences.groupby(['patchID'])['speciesId'].apply(list).reset_index()
data_sans_doublons = data_sans_doublons.merge(donnees_abiotiques, on='patchID', how='inner')
data_sans_doublons.dropna(inplace = True)
data_sans_doublons.reset_index(drop=True, inplace=True)

In [ ]:
 # Création du modèle
nn = NearestNeighbors(n_neighbors=80)
nn.fit(X2)
# le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
distances, indices = nn.kneighbors(X1)
# Récupération des espèces selon les indices des coordonnées des voisins
result = []
for i in range(len(X1)):
    nearest_species = {}
    for j in range(len(indices[i])):
        index = indices[i][j]
        species_list = data_sans_doublons.loc[index]['speciesId']
        for species in species_list:
            if species in nearest_species:
                nearest_species[species] += 1
            else:
                nearest_species[species] = 1
    selected_species = [species for species, count in nearest_species.items() if count > 80 * 0.2]
    result.append(list(selected_species))

# Traitement pour l'export du CSV au format acceptable par Kaggle
blind['nearest_species'] = result
blind = blind[['Id', 'nearest_species']]
blind = blind.rename(columns={'nearest_species': 'Predicted'})    
blind['Predicted'] = blind['Predicted'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace(',', ''))
blind.to_csv("80voisinsabiotiqueproba02(test).csv", index=False, sep=',')
